## 处理设备连接

### 自动处理

默认情况下，Nornir 会自动处理设备的连接。这里指的意思是 Nornir 会自动连接到设备，执行完成任务后再退出设备。

In [1]:
from nornir import InitNornir
from nornir_utils.plugins.functions import print_result
from nornir_napalm.plugins.tasks import napalm_get

In [2]:
nr = InitNornir(config_file="handling_connections/config.yaml")
r1 = nr.filter(name="rt01")

r = r1.run(
    task=napalm_get,
    getters=["facts"]
)

In [3]:
print_result(r)

napalm_get**********************************************************************
* rt01 ** changed : False ******************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'Unknown',
             'hostname': 'R1',
             'interface_list': [],
             'model': 'Unknown',
             'os_version': 'Unknown',
             'serial_number': [],
             'uptime': -1,
             'vendor': 'Huawei'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


### 手动处理

在某些情况下，可能需要手动管理设备的连接，让用户来决定什么时候连接到设备上，什么时候和设备断开连接。

这时候可以使用 `open_connection`、`close_connection`、`close_connections` 和 `Nornir.close_connections` 这几个方法来实现：

In [4]:
def task_manages_connection_manually(task):
    print(f"开始连接：{task.host.name}")
    task.host.open_connection("napalm", configuration=task.nornir.config)
    r = task.run(
        task=napalm_get,
        getters=["facts"]
    )
    print(f"连接成功: {not r[0].failed}")
    task.host.close_connection("napalm")

nr = InitNornir(config_file="handling_connections/config.yaml")
rtr = nr.filter(name="rt01")
r = rtr.run(
    task=task_manages_connection_manually,
)

开始连接：rt01
连接成功: True


In [5]:
print_result(r)

task_manages_connection_manually************************************************
* rt01 ** changed : False ******************************************************
vvvv task_manages_connection_manually ** changed : False vvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_get ** changed : False --------------------------------------------- INFO
{ 'facts': { 'fqdn': 'Unknown',
             'hostname': 'R1',
             'interface_list': [],
             'model': 'Unknown',
             'os_version': 'Unknown',
             'serial_number': [],
             'uptime': -1,
             'vendor': 'Huawei'}}
^^^^ END task_manages_connection_manually ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


### 指定连接参数

使用 `open_connection` 时，可以指定所需要的任何参数，如果没有指定或者如果让 nornir 自动打开设备连接，nornir 将会从主机清单中读取这些连接参数。

在主机清单中的 `connection_options` 中指定设备的连接方式，然后在 `extras` -> `optional_args` 中添加连接插件的额外参数，连接参数通常是下面这种格式：

In [6]:
# %load handling_connections/inventory/test-hosts.yaml
dev1.group_1:
    port: 22
    hostname: 192.168.56.20
    username: username
    password: password
    platform: ios
    connection_options:
        netmiko:
            port: 22
            hostname:
            username: user
            password: pass
            platform: ios
            extras:
                optional_args:
                    secret: secret
                    session_log: path/to/save_log
        
        napalm:
            platform: ios
            extras:
                optional_args:
                    path: path/to/save_log
        
        dummy:
            hostname: dummy_from_host
            port:
            username:
            password:
            platform:
            extras:
                blah: from_host